In [1]:
import cobra
import cobrakbase
from cobrakbase.core.converters import KBaseFBAModelToCobraBuilder

cobrakbase 0.1.9


In [2]:
kbase = cobrakbase.KBaseAPI(YOUR_LOGIN_TOKEN)

In [3]:
ws = "filipeliu:narrative_1561151919387"

In [4]:
fbamodel_json = kbase.get_object('GCF_000005845.2.gf.mdl', ws)
fbamodel = cobrakbase.core.model.KBaseFBAModel(fbamodel_json)

In [5]:
builder = KBaseFBAModelToCobraBuilder(fbamodel)

In [6]:
model = builder.build()

In [7]:
model

Name,GCF_000005845.2.mdl
Memory address,0x0131e9faf28
Number of metabolites,1287
Number of reactions,1566
Number of groups,0
Objective expression,1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711
Compartments,"c0, e0"


# Run COBRA optimization
Notice the growth rate biomass (*bio1_biomass*) is at 298. Without a specified media all extracellular Exchanges are set to (-1000, 1000)

In [8]:
model.summary()

IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  -----------------
cpd00438_e0    1e+03  cpd00009_e0    1e+03  bio1_biomass  298
cpd00023_e0    1e+03  cpd00011_e0    1e+03
cpd00024_e0    1e+03  cpd00013_e0    1e+03
cpd00080_e0    1e+03  cpd00047_e0    1e+03
cpd00106_e0    1e+03  cpd00122_e0    1e+03
cpd00276_e0    1e+03  cpd00222_e0    1e+03
cpd00007_e0  924      cpd00226_e0    1e+03
cpd00041_e0  718      cpd00129_e0  945
cpd00079_e0  500      cpd00137_e0  837
cpd00367_e0  478      cpd00067_e0  692
cpd15352_e0  414      cpd00036_e0  596
cpd00161_e0  281      cpd01015_e0  585
cpd00082_e0  202      cpd15353_e0  414
cpd00107_e0  137      cpd00249_e0  338
cpd00156_e0  105      cpd11416_c0  298
cpd00054_e0   96      cpd00159_e0  251
cpd00039_e0   85.2    cpd00176_e0  225
cpd00051_e0   73.6    cpd00012_e0  130
cpd00322_e0   72.2    cpd00092_e0   82.6
cpd00311_e0   49.9    cpd00108_e0   80.3
cpd00066_e0   46.1    cpd00033_e0   47
cpd03847_e0  

In [9]:
media_json = kbase.get_object('Carbon-D-Glucose', ws)
media = cobrakbase.core.KBaseBiochemMedia(media_json)
media

In [10]:
media.get_media_constraints()

{'cpd00149_e0': (-100, 100),
 'cpd00099_e0': (-100, 100),
 'cpd00067_e0': (-100, 100),
 'cpd00063_e0': (-100, 100),
 'cpd00058_e0': (-100, 100),
 'cpd00048_e0': (-100, 100),
 'cpd00034_e0': (-100, 100),
 'cpd00030_e0': (-100, 100),
 'cpd00013_e0': (-100, 100),
 'cpd00009_e0': (-100, 100),
 'cpd00001_e0': (-100, 100),
 'cpd00007_e0': (-10, 100),
 'cpd00205_e0': (-100, 100),
 'cpd00254_e0': (-100, 100),
 'cpd00971_e0': (-100, 100),
 'cpd10515_e0': (-100, 100),
 'cpd10516_e0': (-100, 100),
 'cpd11574_e0': (-100, 100),
 'cpd00244_e0': (-100, 100),
 'cpd00027_e0': (-5, 100)}

In [11]:
model = builder.with_media(media).build()

In [12]:
model.summary()

IN FLUXES             OUT FLUXES             OBJECTIVES
--------------------  ---------------------  -------------------
cpd00013_e0  5.98     cpd00001_e0  17.8      bio1_biomass  0.697
cpd00027_e0  5        cpd00067_e0   4.86
cpd00009_e0  0.566    cpd00011_e0   2.54
cpd00048_e0  0.155    cpd11416_c0   0.697
cpd10515_e0  0.00648  cpd00007_e0   0.02
cpd00030_e0  0.00216  cpd15378_e0   0.00216
cpd00034_e0  0.00216
cpd00058_e0  0.00216
cpd00063_e0  0.00216
cpd00099_e0  0.00216
cpd00149_e0  0.00216
cpd00205_e0  0.00216
cpd00254_e0  0.00216
cpd10516_e0  0.00216


In [13]:
model.genes[:4]

[<Gene b3177 at 0x131e2beba8>,
 <Gene b2947 at 0x131e24f080>,
 <Gene b0125 at 0x131e24f0b8>,
 <Gene b0775 at 0x131e24f0f0>]

In [14]:
model.genes[0]

Gene identifier,b3177
Name,b3177
Memory address,0x0131e2beba8
Functional,True
In 2 reaction(s),"rxn02201_c0, rxn02200_c0"


## Gene Annotation
The model object does not store any annotation for the genes. We can retreive gene data from the Genome object

In [16]:
model.genes[0].annotation

{'sbo': 'SBO:0000243'}

KBase models are attached to a genome object (*genome_ref*)

In [18]:
fbamodel.data['genome_ref']

'45384/3/1'

We get the object (*id/name*) and workspace (*id*) with *get_object_info_from_ref*

In [20]:
ref_info = kbase.get_object_info_from_ref(fbamodel.data['genome_ref'])
print(ref_info.id, ref_info.workspace_id)

GCF_000005845.2.RAST1 filipeliu:narrative_1561151919387


In [21]:
genome_json = kbase.get_object('GCF_000005845.2.RAST1', 'filipeliu:narrative_1561151919387')
genome = cobrakbase.core.KBaseGenome(genome_json)
genome

In [22]:
genome.data['scientific_name']

'Escherichia coli str. K-12 substr. MG1655'

In [23]:
model = builder.with_media(media).with_genome(genome).build()

In [24]:
model.genes[0].annotation

{'sbo': 'SBO:0000243',
 'ncbiprotein': 'NP_417644.4',
 'ncbigene': '947691',
 'ecogene': 'EG50011'}